In [1]:
# Import der notwendigen Bibliotheken
import pandas as pd
import numpy as np
import pandas_datareader.data as web
import scipy.optimize as sco
from scipy import stats
import matplotlib.pyplot as plt
%matplotlib inline
!pip install yfinance
import yfinance as yf
from datetime import datetime

In [9]:
#download stock portfolio data

yf.pdr_override()
tickers = ['MMM', 'TSLA', 'CSCO', 'V', 'GE', 'AMZN']
start = datetime(2018, 1, 1)
end = datetime(2021, 12, 31)

df = pd.DataFrame([web.get_data_yahoo(ticker, start, end)['Adj Close'] for ticker in tickers]).T
df.columns = tickers
df

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


,MMM,TSLA,CSCO,V,GE,AMZN
Date,,,,,,
2018-01-02,199.068909,21.368668,33.135647,110.811302,103.273834,59.450500
2018-01-03,199.060486,21.150000,33.399975,111.914474,104.250267,60.209999
2018-01-04,201.662460,20.974667,33.494480,112.330582,106.432907,60.479500
2018-01-05,203.233826,21.105333,33.958374,115.020775,106.490356,61.457001
2018-01-08,202.574844,22.427334,34.310574,115.485260,104.996986,62.343498
...,...,...,...,...,...,...
2021-12-23,167.852890,355.666656,59.905960,214.993698,73.065529,171.068497
2021-12-27,169.512512,364.646667,61.002510,215.996109,73.547447,169.669495
2021-12-28,170.414276,362.823334,61.108315,216.412949,74.076012,170.660995


In [10]:
# Definition calc_portfolio_perf

def calc_portfolio_perf(weights, mean_returns, cov, rf, alpha, days, short):
    portfolio_return = np.sum(mean_returns * weights) * days
    portfolio_std = np.sqrt(np.dot(weights.T, np.dot(cov, weights))) * np.sqrt(days)
    sharpe_ratio = (portfolio_return - rf) / portfolio_std
    portfolio_var = abs(portfolio_return - (portfolio_std * stats.norm.ppf(1 - alpha)))
    return portfolio_return, portfolio_std, sharpe_ratio, portfolio_var

In [11]:
# simulate_random_portfolios

def simulate_random_portfolios(num_portfolios, mean_returns, cov, rf, alpha, days, short):
    results_matrix = np.zeros((len(mean_returns)+4, num_portfolios))
    for i in range(num_portfolios):
        if short == 'yes':
            while True:
                weights = np.random.uniform(low = -1,high = 1,size = 5)
                if 1-np.sum(weights) <= 1 and 1-np.sum(weights) >= -1:
                    weights = np.insert(weights, 0, [1-np.sum(weights)])
                    break
        portfolio_return, portfolio_std, sharpe_ratio, portfolio_var = calc_portfolio_perf(weights, mean_returns, cov, rf, alpha, days, short)
        results_matrix[0,i] = portfolio_return
        results_matrix[1,i] = portfolio_std
        results_matrix[2,i] = sharpe_ratio
        results_matrix[3,i] = portfolio_var
        #iterate through the weight vector and add data
        for j in range(len(weights)):
            results_matrix[j+4,i] = weights[j]
            
    results_df = pd.DataFrame(results_matrix.T,columns=['portfolio_return', 'portfolio_std', 'sharpe_ratio', 'portfolio_var'] + [ticker for ticker in tickers])
        
    return results_df

In [12]:
#mit shortselling

mean_returns = np.log1p(df.pct_change()).mean()
cov = df.pct_change().cov()
num_portfolios = 100000
rf = 0
days = 252
alpha = 0.05
short = 'yes'

results_frame_s = simulate_random_portfolios(num_portfolios, mean_returns, cov, rf, alpha, days, short)
results_frame_s

,portfolio_return,portfolio_std,sharpe_ratio,portfolio_var,MMM,TSLA,CSCO,V,GE,AMZN
0,0.279521,0.456850,0.611845,0.471930,-0.815936,0.234165,0.132649,0.466700,0.787590,0.194831
1,0.526564,0.739179,0.712363,0.689278,-0.784774,0.912893,0.612487,0.239625,0.829972,-0.810203
2,0.465080,0.557220,0.834643,0.451465,0.839900,0.715180,-0.983799,-0.307526,-0.011044,0.747290
3,0.630104,0.486626,1.294843,0.170324,-0.977987,0.379515,0.683019,0.970045,-0.205916,0.151324
4,0.313750,0.684791,0.458169,0.812631,0.298314,0.884583,0.168402,-0.103173,0.700103,-0.948228
...,...,...,...,...,...,...,...,...,...,...
995,-0.269785,0.561694,-0.480306,1.193688,0.217162,-0.171616,0.021621,0.974090,0.855434,-0.896691
996,0.114188,0.359842,0.317328,0.477699,-0.554906,-0.328618,0.417479,0.689250,0.168027,0.608768
997,0.505869,0.503351,1.005003,0.322069,0.753426,0.546028,0.729198,-0.455973,-0.763065,0.190386
998,-0.206520,0.455710,-0.453182,0.956096,0.278991,-0.154085,0.411747,0.415680,0.671478,-0.623810


## Betrachtung der Ergebnisse 

In [13]:
#Position des Portfolios mit höchster sharp ratio
max_sharpe_port_s = results_frame_s.iloc[results_frame_s['sharpe_ratio'].idxmax()]

#Position des Portfolios mit geringster std
min_vol_port_s = results_frame_s.iloc[results_frame_s['portfolio_std'].idxmin()]

#Position des Portfolios mit minimalem var

min_VaR_port_s = results_frame_s.iloc[results_frame_s['portfolio_var'].idxmin()]

#Aus jeder Liste ein Dataframe erstellen
max_sharpe_port_s_t = max_sharpe_port_s.to_frame().T
min_vol_port_s_t = min_vol_port_s.to_frame().T
min_VaR_port_s_t = min_VaR_port_s.to_frame().T

#Datensatz  'portfolio aus den Portfolios erstellen
portfolios = pd.DataFrame().append([ min_vol_port_s_t, min_VaR_port_s_t, max_sharpe_port_s_t,])
portfolios = portfolios.reset_index()

portfolios


,index,portfolio_return,portfolio_std,sharpe_ratio,portfolio_var,MMM,TSLA,CSCO,V,GE,AMZN
0,664,0.052915,0.246686,0.214503,0.352848,0.703383,-0.026279,-0.079604,-0.138937,0.020468,0.520969
1,54,0.712618,0.518928,1.373250,0.140943,-0.827466,0.446623,0.745429,0.415231,-0.357955,0.578137
2,54,0.712618,0.518928,1.373250,0.140943,-0.827466,0.446623,0.745429,0.415231,-0.357955,0.578137


In [ ]:
# scatter plot for short portfolio
# create scatter plot coloured by Sharpe Ratio

fig, ax=plt.subplots(figsize=(15,10))
ax.set(xlim=([0.2, 1.2]),ylim=([-1.2, 1.5]))
plt.scatter(results_frame_s.portfolio_std,results_frame_s.portfolio_return,c=results_frame_s.sharpe_ratio,cmap='RdYlBu')
plt.xlabel('Standard Deviation')
plt.ylabel('Returns')
plt.colorbar()